In [2]:
import pandas as pd
import numpy as np
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import load_model
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences


In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# path to  CSV file in  Drive
csv_path = '/content/drive/MyDrive/retailrocket/filtered_sessions.csv'
data = pd.read_csv(csv_path)
data.head()

,timestamp,visitorid,event,itemid,category,global_session_id,interaction_type_id,time_since_last_interaction_logscaled,item_preference_logscaled,item_popularity_logscaled,time_since_session_start_logscaled,session_duration_logscaled
0,2015-09-11 20:49:49.439,0,view,285930,1188,0:START:20150911204949:DUR:00327s,3,0.189860,0.080446,0.229065,0.289316,0.468937
1,2015-09-11 20:52:39.591,0,view,357564,256,0:START:20150911204949:DUR:00327s,3,0.000000,0.000000,0.883486,0.000000,0.000000
2,2015-09-11 20:55:17.175,0,view,67045,333,0:START:20150911204949:DUR:00327s,3,0.000000,0.000000,0.738648,0.000000,0.000000
3,2015-08-07 17:51:44.567,2,view,325215,299,2:START:20150807175144:DUR:01753s,3,0.244409,0.298417,0.684853,0.657039,0.677851
4,2015-08-07 17:53:33.790,2,view,325215,299,2:START:20150807175144:DUR:01753s,3,0.000000,0.000000,0.287558,0.000000,0.000000


In [ ]:
# Load the category prediction model
item_model = load_model('/content/drive/MyDrive/retailrocket/final_item_model.h5')

# Load the Item prediction model
interaction_model = load_model('/content/drive/MyDrive/retailrocket/final_interaction_model.h5')

In [ ]:


# Sort your DataFrame
data['timestamp'] = pd.to_datetime(data['timestamp'])
data.sort_values(by=['visitorid', 'timestamp'], inplace=True)

# Encode the user IDs and  as integers
user_types = data['visitorid'].unique().tolist()
user_encoder = LabelEncoder()
user_encoder.fit(user_types)
data['user_id_encoded'] = user_encoder.transform(data['visitorid'])


# Encode the item IDs and  as integers
item_types = data['itemid'].unique().tolist()
item_encoder = LabelEncoder()
item_encoder.fit(item_types)

# transform data with this updated encoder
data['itemid_encoded'] = item_encoder.transform(data['itemid'])

#  Encode categorie
category_types = data['category'].unique().tolist()
category_encoder = LabelEncoder()
category_encoder.fit(category_types)
data['category_encoded'] = category_encoder.transform(data['category'])


# Encode interactions
interaction_types = data['interaction_type_id'].unique().tolist()
interaction_encoder = LabelEncoder()
interaction_encoder.fit(interaction_types)

# transform  data with this updated encoder
data['interaction_type_encoded'] = interaction_encoder.transform(data['interaction_type_id'])

In [5]:
# define max length based on data
max_sequence_length = data.groupby(['visitorid', 'global_session_id']).size().max()

In [ ]:
# function to predict next interaction type
def predict_next_interaction_for_users(user_ids, data, interaction_encoder, interaction_model, user_encoder, max_sequence_length, num_predictions=None):
    """
    Predicts the next interaction types for specific user IDs for a specified number of predictions or until the first end-of-session label is predicted.

    Args:
    - user_ids: List of user IDs for whom to predict the next interaction types.
    - data: The DataFrame containing all interactions.
    - interaction_encoder: Fitted LabelEncoder for interaction type encoding.
    - interaction_model: Trained RNN model for interaction type prediction.
    - user_encoder: Fitted LabelEncoder for user ID encoding.
    - max_sequence_length: Maximum length of sequences used in training.
    - num_predictions: Optional integer specifying the number of predictions to make. If None, predictions continue indefinitely until an end-of-session label.

    Returns:
    - Dictionary where keys are user IDs and values are lists of predicted interaction types until end-of-session or the specified number of predictions.
    """
    user_predictions = {}

    for user_id in user_ids:
        user_data = data[data['visitorid'] == user_id].sort_values(by='timestamp')
        interaction_sequence = user_data['interaction_type_encoded'].values[-max_sequence_length:]

        if len(interaction_sequence) < max_sequence_length:
            interaction_sequence = np.pad(interaction_sequence, (max_sequence_length - len(interaction_sequence), 0), mode='constant', constant_values=(0))

        interaction_sequence = np.array([interaction_sequence])
        user_index = np.array([user_encoder.transform([user_id])[0]])

        predictions = []
        count_predictions = 0

        while num_predictions is None or count_predictions < num_predictions:
            prediction_probs = interaction_model.predict([interaction_sequence, user_index])
            predicted_interaction = np.argmax(prediction_probs[0])
            predicted_interaction_type = interaction_encoder.inverse_transform([predicted_interaction])[0]

            predictions.append(predicted_interaction_type)
            count_predictions += 1

            # Prepare the next prediction's input
            interaction_sequence = np.append(interaction_sequence[:, 1:], [[predicted_interaction]], axis=1)

        user_predictions[user_id] = predictions

    return user_predictions


In [ ]:

# Function to predict next item for a user
def predict_next_item(user_ids, data, item_model, item_encoder, max_sequence_length, num_predictions=None, top_k=5):
    """
    Predicts the next categories for specific user IDs for a specified number of predictions.

    Args:
        user_ids: List of user IDs for whom to predict the next categories.
        data: The DataFrame containing all interactions.
        item_model: Trained RNN model for category prediction.
        item_encoder: Fitted LabelEncoder for category encoding.
        max_sequence_length: Maximum length of sequences used in training.
        num_predictions: Optional integer specifying the number of predictions to make.
        top_k: Number of top probabilities to consider for sampling (default: 5)

    Returns:
        - Dictionary where keys are user IDs and values are lists of predicted categories.
    """
    user_predictions = {}

    for user_id in user_ids:
        user_data = data[data['visitorid'] == user_id].sort_values(by='timestamp')
        item_sequence = user_data[['itemid_encoded', 'category_encoded', 'item_popularity_logscaled']].values[-max_sequence_length:]

        if len(item_sequence) < max_sequence_length:
            padding = np.zeros((max_sequence_length - len(item_sequence), 3))
            item_sequence = np.vstack((padding, item_sequence))

        item_sequence = np.array([item_sequence])  # Shape: (1, sequence_length, 3)

        predictions = []
        count_predictions = 0

        while num_predictions is None or count_predictions < num_predictions:
            prediction_probs = item_model.predict(item_sequence)

            # Top-k Sampling
            top_probs = prediction_probs[0].argsort()[-top_k:]  # Get indices of top k probabilities (descending order)
            probs = prediction_probs[0][top_probs]  # Get corresponding top k probabilities

            # Sample from the top k probabilities
            predicted_item_index = np.random.choice(top_probs, p=probs / probs.sum())
            predicted_item = item_encoder.inverse_transform([predicted_item_index])[0]

            predictions.append(predicted_item)
            count_predictions += 1

            # Update the sequence with the sampled category
            new_feature = [predicted_item_index, 0, 0]  # Assuming 0 for category and popularity since we do not have this info for predictions
            item_sequence = np.append(item_sequence[:, 1:], [[new_feature]], axis=1)

        user_predictions[user_id] = predictions

    return user_predictions




In [ ]:
user_ids_to_predict = [1384802]

num_predictions = 5  # Number of category predictions to generate for each user

# Call the function
predicted_items = predict_next_item(
    user_ids=user_ids_to_predict,
    data=data,
    item_model=item_model,
    item_encoder=item_encoder,
    max_sequence_length=max_sequence_length,
    num_predictions=num_predictions
)

# Display the results
for user_id, predictions in predicted_items.items():
    print(f"Predicted categories for user {user_id}: {predictions}")

1/1 [==============================] - 0s 59ms/step
Predicted categories for user 1384802: [57578, 450135, 450135, 7943, 7943]


In [ ]:
# Function to sample and generate time stamps for user interactions
def generate_timestamps_for_users(user_ids, data, num_predictions=None):
    """
    Generate timestamps for predicted interactions based on historical interaction time distributions for multiple users.

    Args:
    - user_ids: List of user IDs for whom to generate timestamps.
    - data: DataFrame containing all interactions.
    - num_predictions: Number of timestamps to generate. If None, predictions continue indefinitely.

    Returns:
    - Dictionary where keys are user IDs and values are lists of generated timestamps.
    """
    user_timestamps = {}

    for user_id in user_ids:
        user_data = data[data['visitorid'] == user_id].sort_values(by='timestamp')
        timestamps = pd.to_datetime(user_data['timestamp']).values

        # Handle case where there is insufficient data to calculate time differences
        if len(timestamps) < 2:
            # If only one timestamp or none, assume a default time diff
            time_diffs = [60]  # Default to 60s
            last_timestamp = timestamps[-1] if timestamps.size > 0 else pd.Timestamp.now()
        else:
            time_diffs = np.diff(timestamps).astype('timedelta64[s]').astype(int)
            last_timestamp = timestamps[-1]

        generated_timestamps = []
        count_predictions = 0

        while num_predictions is None or count_predictions < num_predictions:
            time_diff = np.random.choice(time_diffs)
            new_timestamp = last_timestamp + np.timedelta64(time_diff, 's')
            generated_timestamps.append(new_timestamp)
            last_timestamp = new_timestamp
            count_predictions += 1

        user_timestamps[user_id] = generated_timestamps

    return user_timestamps




In [ ]:
# Usage example
user_ids_to_predict = np.random.choice(data['visitorid'].unique(), size=5, replace=False)  # Select 5 random users
num_predictions = 5  # Number of timestamps to generate for each user
predicted_timestamps = generate_timestamps_for_users(user_ids_to_predict, data, num_predictions)
print("Predicted timestamps for each user:", predicted_timestamps)

Predicted timestamps for each user: {756545: [numpy.datetime64('2015-08-30T04:58:58.831000000'), numpy.datetime64('2015-08-30T05:20:20.831000000'), numpy.datetime64('2015-08-30T05:23:55.831000000'), numpy.datetime64('2015-08-30T05:26:50.831000000'), numpy.datetime64('2015-08-30T05:30:25.831000000')], 628540: [numpy.datetime64('2015-05-25T04:13:49.788000000'), numpy.datetime64('2015-05-25T04:15:41.788000000'), numpy.datetime64('2015-05-25T04:19:51.788000000'), numpy.datetime64('2015-05-25T04:21:43.788000000'), numpy.datetime64('2015-05-25T04:21:52.788000000')], 1363492: [numpy.datetime64('2015-05-19T18:12:50.882000000'), numpy.datetime64('2015-05-19T18:12:50.882000000'), numpy.datetime64('2015-05-19T18:12:50.882000000'), numpy.datetime64('2015-05-19T18:13:12.882000000'), numpy.datetime64('2015-05-19T18:34:46.882000000')], 1385030: [numpy.datetime64('2015-06-23T00:00:27.933000000'), numpy.datetime64('2015-06-23T00:02:24.933000000'), numpy.datetime64('2015-06-23T00:04:21.933000000'), nump

In [ ]:
# Define user IDs predict next interactions
unique_user_ids = data['visitorid'].unique()
user_ids_to_predict = np.random.choice(unique_user_ids, size=5, replace=False)

# Call the function
predicted_interactions = predict_next_interaction_for_users(
    user_ids=user_ids_to_predict,
    data=data,
    interaction_encoder=interaction_encoder,
    interaction_model=interaction_model,
    user_encoder=user_encoder,
    max_sequence_length=max_sequence_length,
    num_predictions=5  # Predict up to 10 interactions per user
)

# Print predicted interactions
for user_id, interactions in predicted_interactions.items():
    print(f"User ID {user_id}: Predicted interactions: {interactions}")


1/1 [==============================] - 0s 34ms/step
User ID 906934: Predicted interactions: [3, 3, 3, 3, 3]
User ID 148632: Predicted interactions: [3, 3, 3, 3, 3]
User ID 108759: Predicted interactions: [1, 2, 2, 2, 2]
User ID 1396860: Predicted interactions: [3, 3, 3, 3, 3]
User ID 1364998: Predicted interactions: [3, 3, 3, 3, 3]


In [6]:
# Funtion to integrate other functions for prediciting item ids, interactions and time stamps
def get_predictions_for_users(user_ids, data, item_model, item_encoder, interaction_model, interaction_encoder, user_encoder, max_sequence_length, num_predictions_per_user=None):
    all_predictions = []

    for user_id in user_ids:

        num_predictions = num_predictions_per_user.get(user_id, np.random.randint(2, 20))

        # Generate predictions for each type
        timestamps_dict = generate_timestamps_for_users([user_id], data, num_predictions)
        items_dict = predict_next_item([user_id], data, item_model, item_encoder, max_sequence_length, num_predictions, top_k=5)
        interactions_dict = predict_next_interaction_for_users(
                            user_ids=[user_id],
                            data=data,
                            interaction_encoder=interaction_encoder,
                            interaction_model=interaction_model,
                            user_encoder=user_encoder,
                            max_sequence_length=max_sequence_length,
                            num_predictions=num_predictions
                        )

        # Extract the lists from the dictionaries
        timestamps = timestamps_dict[user_id]
        items = items_dict[user_id]
        interactions = interactions_dict[user_id]


        # Debugging print statements
        print(f"Timestamps for user {user_id}: {timestamps}")
        print(f"Items for user {user_id}: {items}")
        print(f"Interactions for user {user_id}: {interactions}")

        # Combine predictions into tuples
        predictions = list(zip([user_id]*num_predictions, items, interactions, timestamps))
        all_predictions.extend(predictions)

    # Create DataFrame from the combined predictions
    df = pd.DataFrame(all_predictions, columns=['UserID', 'ItemID', 'InteractionType', 'Timestamp'])
    return df

In [ ]:

# List to store each iteration's DataFrame
all_iterations_data = []

for i in range(10):
    user_ids = np.random.choice(data['visitorid'].unique(), size=100, replace=False)
    num_predictions_per_user = {user_id: np.random.randint(2, 20) for user_id in user_ids}

    generated_data = get_predictions_for_users(user_ids, data, item_model, item_encoder, interaction_model, interaction_encoder, user_encoder, max_sequence_length, num_predictions_per_user)
    all_iterations_data.append(generated_data)

# Concatenate all DataFrames from each iteration into one DataFrame
final_generated_data = pd.concat(all_iterations_data, ignore_index=True)

#  display the result
print(final_generated_data.head())


Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 86ms/step
Timestamps for user 1271573: [numpy.datetime64('2015-08-19T20:30:01.732000000'), numpy.datetime64('2015-08-19T20:30:49.732000000'), numpy.datetime64('2015-08-19T20:31:18.732000000'), numpy.datetime64('2015-08-19T20:32:06.732000000'), numpy.datetime64('2015-08-19T20:32:35.732000000'), numpy.datetime64('2015-08-19T20:33:04.732000000'), numpy.datetime64('2015-08-19T20:33:52.732000000'), numpy.datetime64('2015-08-19T20:34:21.732000000'), numpy.datetime64('2015-08-19T20:34:50.732000000'), numpy.datetime64('2015-08-19T20:44:48.732000000'), numpy.datetime64('2015-08-19T20:45:17.732000000'), numpy.datetime64('2015-08-19T20:46:05.732000000'), numpy.datetime64('2015-08-19T20:56:03.732000000'), numpy.datetime64('2015-08-19T20:56:32.732000000'), numpy.datetime64('2015-08-19T21:06:30.732000000'), numpy.datetime64('2015-08-19T21:16:28.732000000')]
Items for user 1271573: [218856, 346213, 346213, 34

In [ ]:
final_generated_data.to_csv('/content/drive/MyDrive/retailrocket/final_generated_data.csv')

print(f"Data has been saved drive")

Data has been saved drive
